In [42]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pprint
import json
import urllib
import requests

# Get Covid data from api.covidtracking.com
1. suppose 'data' is the retrieved json, then it has keys ['links', 'meta', 'data']<br>
2. For us/daily, data['data'] stores daily number of cases, with keys ['date', 'states', 'cases', 'testing', 'outcomes']<br>
['date'] is the date in YYYY-MM-DD<br>
['state'] is the number of states<br>
['cases'] has a single key ['total'] with ['value'] (raw number) and ['calculated'] (some statistics)<br>
['testing'] is structured the same way as ['cases']<br>
['outcomes']<br>
3. For state/daily, data['data'] has keys ['date', 'state', 'meta', 'cases', 'tests', 'outcomes']

In [106]:
def get_state_codes():
    l = 'https://api.covidtracking.com/v2/states.json'
    with urllib.request.urlopen(l) as url:
        link_json = json.load(url)
    pd.DataFrame(link_json['data']).to_csv('./data/states.csv')

In [128]:
def retrieve_json(link):
    try:
        with urllib.request.urlopen(link) as url:
            link_json = json.load(url)
        filename = './data/' + link.split('v2/')[1].replace('/', '_')
        with open(filename, 'w') as f:
            json.dump(link_json, f)
        print('saved ' + filename)
        if filename != './data/us_daily.json':
            covid_files.append(filename)
    except:
        print('retrieval failed ' + link)

In [113]:
def get_all_json():
    global state_codes
    covidtracking_api = 'https://api.covidtracking.com/v2/'
    
    retrieve_json(covidtracking_api+'us/daily.json')
    
    states_codes = pd.read_csv('./data/states.csv')['state_code'].str.lower()
    for state in states_codes:
        retrieve_json(covidtracking_api + 'states/' + state + '/daily/simple.json')

In [169]:
def clean_df_from_json(filename):
    with open(filename, 'r') as f:
        temp = json.load(f)
    temp = pd.DataFrame(temp['data'])
    
    if filename != './data/us_daily.json':
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total'])
        temp['cases_confirmed'] = temp['cases'].apply(lambda x: x['confirmed'])
        temp['cases_probable'] = temp['cases'].apply(lambda x: x['probable'])
        temp['tests_total'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['total'])
        temp['tests_positive'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['positive'])
        temp = temp.drop(['cases', 'tests'], axis=1)
        temp.to_csv(filename.replace('json', 'csv'))
        print('saved ' + filename.replace('json', 'csv'))
    else:
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total']['value'])
        temp['cases_percent'] = temp['cases'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['cases_increase'] = temp['cases'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['cases_7d_change'] = temp['cases'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp['testing_total'] = temp['testing'].apply(lambda x: x['total']['value'])
        temp['testing_percent'] = temp['testing'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['testing_increase'] = temp['testing'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['testing_7d_change'] = temp['testing'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp = temp.drop(['cases', 'testing'], axis=1)
        temp.to_csv('./data/us_daily.csv')
        print('saved us_daily.csv')

In [170]:
def clean_all_df():
    clean_df_from_json('./data/us_daily.json')
    for file in covid_files:
        clean_df_from_json(file)

In [107]:
get_state_codes()

In [93]:
covid_files = []
get_all_json()

saved states_al_daily_simple.json
saved states_ak_daily_simple.json
saved states_az_daily_simple.json
saved states_ar_daily_simple.json
saved states_ca_daily_simple.json
saved states_co_daily_simple.json
saved states_ct_daily_simple.json
saved states_de_daily_simple.json
saved states_dc_daily_simple.json
saved states_fl_daily_simple.json
saved states_ga_daily_simple.json
saved states_hi_daily_simple.json
saved states_id_daily_simple.json
saved states_il_daily_simple.json
saved states_in_daily_simple.json
saved states_ia_daily_simple.json
saved states_ks_daily_simple.json
saved states_ky_daily_simple.json
saved states_la_daily_simple.json
saved states_me_daily_simple.json
saved states_md_daily_simple.json
saved states_ma_daily_simple.json
saved states_mi_daily_simple.json
saved states_mn_daily_simple.json
saved states_ms_daily_simple.json
saved states_mo_daily_simple.json
saved states_mt_daily_simple.json
saved states_ne_daily_simple.json
saved states_nv_daily_simple.json
saved states_n

In [171]:
clean_all_df()

saved us_daily.csv
saved states_al_daily_simple.csv
saved states_ak_daily_simple.csv
saved states_az_daily_simple.csv
saved states_ar_daily_simple.csv
saved states_ca_daily_simple.csv
saved states_co_daily_simple.csv
saved states_ct_daily_simple.csv
saved states_de_daily_simple.csv
saved states_dc_daily_simple.csv
saved states_fl_daily_simple.csv
saved states_ga_daily_simple.csv
saved states_hi_daily_simple.csv
saved states_id_daily_simple.csv
saved states_il_daily_simple.csv
saved states_in_daily_simple.csv
saved states_ia_daily_simple.csv
saved states_ks_daily_simple.csv
saved states_ky_daily_simple.csv
saved states_la_daily_simple.csv
saved states_me_daily_simple.csv
saved states_md_daily_simple.csv
saved states_ma_daily_simple.csv
saved states_mi_daily_simple.csv
saved states_mn_daily_simple.csv
saved states_ms_daily_simple.csv
saved states_mo_daily_simple.csv
saved states_mt_daily_simple.csv
saved states_ne_daily_simple.csv
saved states_nv_daily_simple.csv
saved states_nh_daily_si

# Get Covid data (of March 10) from worldometers

In [6]:
wdm = 'https://www.worldometers.info/coronavirus/country/us/'
wdm_page = requests.get(wdm)

In [12]:
df = pd.read_html(wdm_page.content)[1]
df.columns.to_list() # check weird column names

['#',
 'USAState',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/ 1M pop',
 'Population',
 'Projections']

In [16]:
# change weird column names
df = df.rename(columns={'Tot\xa0Cases/1M pop': 'TotCasesPer1MPop', 
                        'Deaths/1M pop':'DeathsPer1MPop', 'Tests/ 1M pop': 'TestsPer1MPop'})
df.to_csv('./data/us_0310.csv')

# Get weather data

1. county -> severals zones -> several stations
2. county observations are completely empty
3. county observationStations are also empty

In [100]:
# the county_stations is always empty as returned by the api, so we have another function
# get_state_stations
def get_state_counties(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/zones?area={state_abbr}&type=county'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    county_links = [item['id'] for item in j]
    county_ids = [item['properties']['id'] for item in j]
    county_names = [item['properties']['name'] for item in j]
    county_states = [item['properties']['state'] for item in j]
    county_stations = [item['properties']['observationStations'] for item in j] # empty
    
    df = pd.DataFrame({'link': county_links, 'id': county_ids, 'name': county_names,
                      'state_code': county_states, 'stations': county_stations})
    df['state'] = [state]*len(j)
    return df

In [23]:
# the api does not show any stations for a given county, so we have to get all stations in
# a state and match them to their counties
def get_state_stations(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/stations?state={state_abbr}'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    station_forecasts = []
    for i, item in enumerate(j):
        if 'forecast' in item['properties'].keys():
            station_forecasts.append(item['properties']['forecast'])
        else:
            station_forecasts.append(np.nan)

    station_counties = []
    for i, item in enumerate(j):
        if 'county' in item['properties'].keys():
            station_counties.append(item['properties']['county'].split('/')[-1])
        else:
            station_counties.append(np.nan)
    
    df = pd.DataFrame({'forecast': station_forecasts, 'county': station_counties})
    df = df.dropna(axis=0)
    df = df.drop_duplicates(ignore_index=True)
    
    return df

In [122]:
def get_station_county_pair(state):
    counties_of_state = get_state_counties(state)
    stations_of_state = get_state_stations(state)
    df = pd.merge(cali, cali_station, left_on='id', right_on='county', 
                  suffixes=['_county', '_station'])
    df = df.drop('county', axis=1)
    df.to_csv(f'./data/{state}_station_county_pair.csv')

In [ ]:
def get_state_weather(state):
    station_county_map = pd.read_csv(f'./data/{state}_station_county_pair.csv', index_col=0)

In [ ]:
def get_county_weather(county, station_county_map):
    forecasts = station_county_map[station_county_map['name']==county]['forcast']
    for forecast in forcasts:
        request = requests.get(forecast)
        if request.status_code != 200:
            continue
        else:
            print(forcast, 'api call is successful')
            with urllib.request.urlopen(forcast) as url:
                print(url.)

In [46]:
states = pd.read_csv('./data/states.csv', index_col=0)

In [123]:
get_station_county_pair('California')

https://api.weather.gov/zones?area=CA&type=county api call is successful
https://api.weather.gov/stations?state=CA api call is successful


In [29]:
d = pd.read_csv('./data/California_station_county_pair.csv', index_col=0)

In [38]:
t = d['forecast'][0] + '/observations?limit=10'

In [41]:
d

,link,id,name,state_code,stations,state,forecast
0,https://api.weather.gov/zones/county/CAC001,CAC001,Alameda,CA,[],California,https://api.weather.gov/zones/forecast/CAZ511
1,https://api.weather.gov/zones/county/CAC001,CAC001,Alameda,CA,[],California,https://api.weather.gov/zones/forecast/CAZ508
2,https://api.weather.gov/zones/county/CAC001,CAC001,Alameda,CA,[],California,https://api.weather.gov/zones/forecast/CAZ510
3,https://api.weather.gov/zones/county/CAC003,CAC003,Alpine,CA,[],California,https://api.weather.gov/zones/forecast/CAZ069
4,https://api.weather.gov/zones/county/CAC003,CAC003,Alpine,CA,[],California,https://api.weather.gov/zones/forecast/CAZ072
...,...,...,...,...,...,...,...
210,https://api.weather.gov/zones/county/CAC111,CAC111,Ventura,CA,[],California,https://api.weather.gov/zones/forecast/CAZ550
211,https://api.weather.gov/zones/county/CAC113,CAC113,Yolo,CA,[],California,https://api.weather.gov/zones/forecast/CAZ017
212,https://api.weather.gov/zones/county/CAC115,CAC115,Yuba,CA,[],California,https://api.weather.gov/zones/forecast/CAZ067
213,https://api.weather.gov/zones/county/CAC115,CAC115,Yuba,CA,[],California,https://api.weather.gov/zones/forecast/CAZ016


In [40]:
# Use this to get weather of station
with urllib.request.urlopen(t) as url:
        print(url.)

In [35]:
j3 = json.loads(k3)
j3.keys()

dict_keys(['@context', 'type', 'features'])

In [37]:
j3['features'][0]

{'id': 'https://api.weather.gov/stations/KCCR/observations/2021-03-10T14:53:00+00:00',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-122.05, 37.97]},
 'properties': {'@id': 'https://api.weather.gov/stations/KCCR/observations/2021-03-10T14:53:00+00:00',
  '@type': 'wx:ObservationStation',
  'elevation': {'value': 7, 'unitCode': 'unit:m'},
  'station': 'https://api.weather.gov/stations/KCCR',
  'timestamp': '2021-03-10T14:53:00+00:00',
  'rawMessage': 'KCCR 101453Z 15006KT 10SM FEW060 07/04 A2992 RMK AO2 RAB11E31 SLP117 P0001 60004 T00720044 53002',
  'textDescription': 'Mostly Clear',
  'icon': 'https://api.weather.gov/icons/land/day/few?size=medium',
  'presentWeather': [],
  'temperature': {'value': 7.2,
   'unitCode': 'unit:degC',
   'qualityControl': 'qc:V'},
  'dewpoint': {'value': 4.4,
   'unitCode': 'unit:degC',
   'qualityControl': 'qc:V'},
  'windDirection': {'value': None,
   'unitCode': 'unit:degree_(angle)',
   'qualityControl': 'qc:Z'},
  'windSpeed': 